# Razonamiento de los Agentes
Este código muestra cómo implementar razonamiento en agentes que interactúan con el entorno.

## 1.- Setup inicial

### 1.1- Instalar librerías

In [2]:
#! pip install openai
#! pip install tenacity
#! pip install python-dotenv

### 1.2.- Cargar librerías

In [4]:
import openai
import os
from dotenv import load_dotenv

### 1.3.- Variables de entorno

In [5]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

OpenAI API key: sk-eO...Re5lw
GPT-3.5-Turbo model: gpt-3.5-turbo
GPT-3.5-Turbo-16k model: gpt-3.5-turbo-16k
GPT-4 model: gpt-4


### 2.0.- Ejemplo sin razonamiento 

In [71]:
from llm.openai import generate_text

user_input = "Toma la última letra de cada palabra en 'Larry Page' y concaténalas"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 1: {response}") #Resp: ye

user_input = "Jane programó 3 citas con 5 personas para mañana (martes, 7/9/1972). ¿Cuál es la fecha hace una semana a partir de hoy en formato MM/DD/YYYY?"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 2: {response}") #Resp:  07/01/1972

Respuesta Ejemplo 1: "e"
Respuesta Ejemplo 2: La fecha hace una semana a partir de hoy en formato MM/DD/YYYY es 7/2/1972.


### 2.1.1- Chain Of Thought (CoT) Simple

In [75]:
# CoT Prompt
COT_PROMPT = "Vamos a pensar paso a paso."

user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 1: {response}\n") #Resp: ye

user_input = f"Jane programó 3 citas con 5 personas para mañana (martes, 7/9/1972). ¿Cuál es la fecha hace una semana a partir de hoy en formato MM/DD/YYYY?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 2: {response}\n") #Resp:  07/01/1972

Respuesta Ejemplo 1: 1. L -> La última letra de la palabra "Larry" es "y"
2. P -> La última letra de la palabra "Page" es "e"
3. Concatenando las últimas letras obtenemos "ye"

Por lo tanto, la respuesta es "ye".

Respuesta Ejemplo 2: Para encontrar la fecha hace una semana a partir de hoy, seguimos los siguientes pasos:

1. Tomamos la fecha de hoy, que es 7/9/1972.
2. Restamos una semana a la fecha de hoy. Esto significa restar 7 días a la fecha.
3. Esto nos dará la nueva fecha.

Siguiendo estos pasos:

1. La fecha de hoy es 7/9/1972.
2. Restamos una semana a la fecha de hoy: 7/9/1972 - 7 días = 7/2/1972.
3. Por lo tanto, hace una semana a partir de hoy en el formato MM/DD/YYYY es el 7/2/1972.



### 2.1.2- Chain Of Thought (CoT) + Few Shot

In [88]:
def add_message(messages=[], role="user", input=''):
    messages.append({"role": role, "content": input})

# Inyeccion de mensajes de ejemplo
messages=[]
#Example 1: Math problem
add_message(messages,"user","There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?")
add_message(messages,"assistant"," There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.")
#Example 2: Concadenate problem
add_message(messages,"user","Take the last letters of the words in 'Elon Musk' and concatenate them.")
add_message(messages,"assistant"," The last letter of 'Elon' is 'n'. The last letter of 'Musk' is 'k'. Concatenating them is 'nk'. The answer is nk.")
#Example 3:  CSQA problem
add_message(messages,"user","What do people use to absorb extra ink from a fountain pen? Answer Choices: (a) shirt pocket (b) calligrapher’s hand (c) inkwell (d) desk drawer (e) blotter")
add_message(messages,"assistant","The answer must be an item that can absorb ink. Of the above choices, only blotters are used to absorb ink.So the answer is (e).")
#Example 4:  Date Understanding problem
add_message(messages,"user"," Jane was born on the last day of Feburary in 2001. Today is her 16-year-old birthday. What is the date yesterday in MM/DD/YYYY?")
add_message(messages,"assistant","The last day of February is the 28th, so Jane was born on 02/28/2001. Today is her 16-year old birthday, so today is 02/28/2017. So yesterday was 02/27/2017. So the answer is 02/27/2017.")

# Se ejecuta 1 peticion
user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model,messages=messages)
print(f"Respuesta Ejemplo 1: {response}\n") #Resp: ye

# Se ejecuta 2 peticion
user_input = f"Michael tenía 58 pelotas de golf. El martes perdió 23 pelotas de golf. El miércoles perdió 2 más. ¿Cuántas pelotas de golf tenía al final del miércoles?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model,messages=messages)
print(f"\nRespuesta Ejemplo 2: {response}\n") #Resp: 33

Respuesta Ejemplo 1: Claro, vamos a realizarlo paso a paso:

- La última letra de "Larry" es "y".
- La última letra de "Page" es "e".

Luego, concatenando estas letras obtenemos "ye". Esa sería la respuesta final.


Respuesta Ejemplo 2: Para resolver el problema, vamos a hacer los cálculos paso a paso:

Michael tenía inicialmente 58 pelotas de golf.

El martes perdió 23 pelotas de golf. Entonces, restamos 23 a las 58 pelotas que tenía:

58 - 23 = 35

Después, el miércoles perdió 2 pelotas de golf adicionales. Restamos 2 a las 35 que tenía:

35 - 2 = 33

Entonces, al final del miércoles, Michael tenía 33 pelotas de golf.

La respuesta es que Michael tenía 33 pelotas de golf al final del miércoles.



### 2.1.- Tree Of Thought (ToT)

### 2.1.- Graph Of Thought (GoT)